# **COVID19 Chloropleth Map**

https://towardsdatascience.com/how-to-build-plotly-choropleth-map-with-covid-data-using-pandas-in-google-colab-45951040b8e4
https://pythonmatplotlibtips.blogspot.com/2017/12/plot-on-image-matplotlib-pyplot.html

#Practice: Gathering Data on COVID-19
What are elements of weather and climate that we can measure? 

We can collect data about rainfall, temperature and humidity. This can predict the weather conditions to determine where is the best place for certain activities.

Importing the necessary libraries and packages:

In [1]:
!pip install plotly-geo
!pip install geopandas
!pip install pyshp
!pip install shapely

     |████████████████████████████████| 23.7MB 121kB/s 
     |████████████████████████████████| 1.0MB 31.9MB/s 
     |████████████████████████████████| 6.6MB 46.9MB/s 
     |████████████████████████████████| 15.4MB 128kB/s 
     |████████████████████████████████| 225kB 35.6MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37325 sha256=3d41773ed16efa975dc112138540c34533dacbfb013c75b83289fa5aad067cca
  Stored in directory: /root/.cache/pip/wheels/76/2b/d4/53e6b9a0fb0a9f9f29664cf82605af8bc81d5ab44d987896dd
Successfully built pyshp


In [2]:
# Clone repository
!git clone https://github.com/sidbannet/US-political-analysis.git
%cd ./US-political-analysis
!git submodule init
!git submodule update --remote

Cloning into 'US-political-analysis'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 89 (delta 42), reused 53 (delta 18), pack-reused 0
Unpacking objects: 100% (89/89), done.
/content/US-political-analysis
Submodule 'COVID-19' (https://github.com/CSSEGISandData/COVID-19.git) registered for path 'COVID-19'
Submodule 'stat-tools' (https://github.com/sidbannet/stat-tools.git) registered for path 'stat-tools'
Cloning into '/content/US-political-analysis/COVID-19'...
Cloning into '/content/US-political-analysis/stat-tools'...
Submodule path 'COVID-19': checked out '3048d5312ddb2c54c3a8a91f7b7bff68a87aca78'
Submodule path 'stat-tools': checked out 'bd932b36addc046de367433bf8598c42ec49775c'


In [3]:
from urllib.request import urlopen
import os
import pandas as pd
import plotly.express as px
import json

Ensure that all the libraries are imported!

## **Part 1 - Collection of information**

Let's create our data set of information. First, we have to create a Comma Separated Variables (CSV).

Matplotlib Library is a plotting library for the Python programming language and its numerical mathematics extension NumPy. It provides an object-oriented API for embedding plots into applications using general-purpose GUI toolkits like Tkinter, wxPython, Qt, or GTK. (https://matplotlib.org/)


In [ ]:
## Initialize pandas
df_covid_dead = pd.DataFrame()
## Define folder
us_data_folder = r'COVID-19' \
                 + os.sep + 'csse_covid_19_data' \
                 + os.sep + 'csse_covid_19_time_series' \
                 + os.sep
## Read COVID data
df_covid_dead = pd.read_csv(
    us_data_folder + 'time_series_covid19_deaths_US.csv',
)
df_covid_conf = pd.read_csv(
    us_data_folder + 'time_series_covid19_confirmed_US.csv'
)

In [ ]:
## US COVID data filtering
df_covid = (
    df_covid_dead[df_covid_dead.FIPS > 1000]
)[df_covid_dead.FIPS < 80000].copy()  # This FIPS lies in US voting counties
df_conf = (
    df_covid_conf[df_covid_conf.FIPS > 1000]
)[df_covid_conf.FIPS < 80000].copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



# **Part 2 - Data Analysis**
https://pythonmatplotlibtips.blogspot.com/2017/12/plot-on-image-matplotlib-pyplot.html

In [ ]:
num_days = 14
df_conf_ts = (
    (
        (
            df_conf.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
         ).diff(
             periods=num_days
             ) / 1.0
)
df_dead_ts = (
    (
        (
            df_covid.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key',
          'Population', ]
         ).diff(
             periods=num_days
             ) / 1.0
)

df_conf_ts_cum = (
    (
        (
            df_conf.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
         )
)
df_dead_ts_cum = (
    (
        (
            df_covid.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key',
          'Population', ]
         )
)

df_presentation = pd.DataFrame(
    {
        'FIPS': df_covid.FIPS,
        'State': df_covid.Province_State,
        'County': df_covid.Admin2,        
        'Population': df_covid.Population,
        'Confirmed': df_conf_ts.values[-1],
        'Death': df_dead_ts.values[-1],
        'Death_total': df_dead_ts_cum.values[-1],
        'Confirmed_earlier': df_conf_ts.values[-1 - num_days],
        'Death_earlier': df_dead_ts.values[-1 - num_days],
    },
)

df_presentation = df_presentation[df_presentation.Population > 0]
df_presentation['Confirmed_per_100k'] = df_presentation.Confirmed / df_presentation.Population * 100000
df_presentation['Confirmed_Growth_per_100k'] = (
    df_presentation.Confirmed - df_presentation.Confirmed_earlier
    ) / df_presentation.Population * 100000
df_presentation['Total_Death_per_mil'] = df_presentation.Death_total / df_presentation.Population * 1000000
df_presentation = df_presentation.drop(
    columns=[
        'Confirmed',
        'Death',
        'Death_total',
        'Confirmed_earlier',
        'Death_earlier',
    ]
)

df_presentation['FIPS'] = df_presentation['FIPS'].astype('int64', copy=True)
df_presentation['FIPS'] = df_presentation['FIPS'].astype('str', copy=True)
df_presentation['FIPS'] = df_presentation['FIPS'].str.rjust(5, '0')

df_presentation = df_presentation.set_index('FIPS')

NameError: ignored

In [ ]:

#@title Analysis of data { run: "auto", display-mode: "form" }
#@markdown **Choose with these options:**
Number_of_days = "14" #@param ["7", "14", "21", "28"] {allow-input: true}

num_days = int(Number_of_days)

df_covid_analysis = pd.DataFrame(
    data=df_covid.FIPS
).set_index('FIPS')
df_conf_ts = (
    (
        (
            df_conf.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
         ).diff(
             periods=num_days
             ) / 1.0
)
df_dead_ts = (
    (
        (
            df_covid.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key',
          'Population', ]
         ).diff(
             periods=num_days
             ) / 1.0
)

df_conf_ts_cum = (
    (
        (
            df_conf.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
         )
)
df_dead_ts_cum = (
    (
        (
            df_covid.set_index('FIPS')
        ).T
     ).drop(
         ['iso2', 'iso3', 'UID', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key',
          'Population', ]
         )
)

df_presentation = pd.DataFrame(
    {
        'FIPS': df_covid.FIPS,        
        'State': df_covid.Province_State,
        'County': df_covid.Admin2,        
        'Population': df_covid.Population,
        'Confirmed': df_conf_ts.values[-1],
        'Death': df_dead_ts.values[-1],
        'Death_total': df_dead_ts_cum.values[-1],
        'Confirmed_earlier': df_conf_ts.values[-1 - num_days],
        'Death_earlier': df_dead_ts.values[-1 - num_days],
    },
)

df_grouped = (df_presentation.drop(['County', 'Population', 'FIPS'], axis='columns')).groupby('State').sum()
df_grouped['Population'] = (df_presentation.drop(['County', 'FIPS'], axis='columns')).groupby('State').sum()

df_presentation = df_presentation[df_presentation.Population > 0]
df_presentation['Confirmed_per_100k'] = df_presentation.Confirmed / df_presentation.Population * 100000
df_presentation['Death_per_100k'] = df_presentation.Death / df_presentation.Population * 100000
df_presentation['Confirmed_Growth'] = (df_presentation.Confirmed - df_presentation.Confirmed_earlier) / df_presentation.Population * 100000
df_presentation['Death_Growth'] = (df_presentation.Death - df_presentation.Death_earlier) / df_presentation.Population * 100000
df_presentation['Total_Death_per_mil'] = df_presentation.Death_total / df_presentation.Population * 1000000
df_presentation['Projected_per_mil'] = (
    df_presentation.Confirmed_Growth + df_presentation.Confirmed_per_100k
    )

df_presentation['FIPS'] = df_presentation['FIPS'].astype('int64', copy=True)
df_presentation['FIPS'] = df_presentation['FIPS'].astype('str', copy=True)
df_presentation['FIPS'] = df_presentation['FIPS'].str.rjust(5, '0')

df_presentation = df_presentation.set_index('FIPS')

display(df_grouped)

NameError: ignored

# **Part 3 - Visualising Data**

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
fig = px.choropleth_mapbox(df_presentation.reset_index(), 
                           geojson=counties, locations='FIPS', color='Confirmed_per_100k',
                           color_continuous_scale='jet',
                           range_color=(100, 1500),
                           mapbox_style="carto-positron",
                           hover_name='County',                           
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'Confirmed_per_100k':'Last 2 week data'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
fig = px.choropleth_mapbox(df_presentation.reset_index(), 
                           geojson=counties, locations='FIPS', color='Total_Death_per_mil',                           
                           color_continuous_scale='jet',
                           range_color=(00, 1000),
                           mapbox_style="carto-positron",
                           hover_name='County',                           
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'Total_Death_per_mil':'Death per mil'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# **Part 4 - Importing Data from Google Maps**

Google Maps is an Application Paging Index (API)